# Flask を使った簡易Webアプリ

まずは必要なライブラリをインストール

In [33]:
!pip install -r requirements.txt

2898.53s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


## その１　とりあえず音声ファイルをアップロードできるようにする．

In [42]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Voice Chat App</title>
  </head>
  <body>
    <h1>Voice Chat App</h1>
    <button id="start">開始</button>
    <button id="stop" disabled>停止</button>
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
      document.addEventListener("DOMContentLoaded", () => {
        const startButton = document.getElementById("start");
        const stopButton = document.getElementById("stop");
        const transcriptionElement = document.getElementById("transcription");
        const aiResponseElement = document.getElementById("aiResponse");
        let mediaRecorder;
        let audioChunks = [];

        startButton.addEventListener("click", async () => {
          const stream = await navigator.mediaDevices.getUserMedia({
            audio: true,
          });
          mediaRecorder = new MediaRecorder(stream);

          mediaRecorder.ondataavailable = (event) => {
            audioChunks.push(event.data);
          };

          mediaRecorder.onstop = async () => {
            const audioBlob = new Blob(audioChunks, { type: "audio/webm" });
            audioChunks = [];

            const formData = new FormData();
            formData.append("audio", audioBlob, "recording.webm");

            fetch("/upload", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                transcriptionElement.textContent =
                  data.text || "認識できませんでした。";
                aiResponseElement.textContent =
                  data.ai_response || "AIの応答なし。";
              })
              .catch((error) => {
                console.error("Upload failed:", error);
                transcriptionElement.textContent = "エラーが発生しました。";
                aiResponseElement.textContent = "";
              });
          };

          mediaRecorder.start();
          startButton.disabled = true;
          stopButton.disabled = false;
        });

        stopButton.addEventListener("click", () => {
          mediaRecorder.stop();
          startButton.disabled = false;
          stopButton.disabled = true;
        });
      });
    </script>
  </body>
</html>


Overwriting static/index.html


In [ ]:
%%writefile app01.py

from flask import Flask, request, jsonify, send_from_directory
import os

app = Flask(__name__, static_folder="static")  

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app01.py


In [36]:
!python app01.py

2909.93s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


 * Serving Flask app 'app01'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
^C


## その２　アップロードした音声ファイルをWavファイルに変換

In [ ]:
%%writefile app02.py

from flask import Flask, request, jsonify, render_template
import os
import subprocess

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")  # フロントエンドのHTMLを表示

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    convert_webm_to_wav(audio_path, "uploads/output.wav")
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})


def convert_webm_to_wav(input_path, output_path):
    command = [
        "ffmpeg",
        "-i", input_path,  # 入力ファイル
        "-ar", "16000",  # サンプリングレート 16kHz
        "-ac", "1",  # モノラル変換
        "-preset", "ultrafast",  # 速度最優先
        output_path
    ]
    subprocess.run(command, check=True)

# 使い方

if __name__ == "__main__":
    app.run(debug=True)


Writing app02.py


In [ ]:
!python app02.py

 * Serving Flask app 'app02'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [26/Feb/2025 22:49:28] "GET / HTTP/1.1" 200 -
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 

変換に少々時間取られるな．．．

## その３　アップロードを直接Wavファイルにする．

変換に少々時間がかかるのが気になるので，アップロードの段階で直接Wavファイルをアップできないか探ってみたら，Record.jsなるものがあるようだ．
https://github.com/mattdiamond/Recorderjs
Recorder.jsをダウンロードして
これをhtmlに組み込んでみる．

In [52]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>
    <!-- <script src="recorder.js"></script> --> 
    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;
                

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                    document.getElementById("aiResponse").textContent =
                    data.ai_response || "AIの応答なし。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index.html


In [38]:
!python app01.py

3924.77s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


 * Serving Flask app 'app01'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [27/Feb/2025 06:07:04] "GET / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-packages/flask/app.py", line 1536, in __call__
    return self.wsgi_app(environ, start_response)
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-packages/flask/app.py", line 1514, in wsgi_app
    response = self.handle_exception(e)
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/fujinohidenori/dev/project/venv/lib/python3.13/site-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/fujinohidenori/dev/

CORSの問題でアップロードで弾かれているようだ．．．

flask_corsを使って，サーバの側でCORS問題を無視するように設定する．

In [ ]:
!pip install flask_cors

In [40]:
%%writefile app03.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app03.py


In [41]:
!python app03.py

4051.27s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


 * Serving Flask app 'app03'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [27/Feb/2025 06:09:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2025 06:09:15] "POST /upload HTTP/1.1" 200 -
^C


よし，とりあえず問題は解決した．
ハマった理由は，fetchのインタフェースをフォルダ名と勘違いしていたこと．つまり，app.routeでは/uploadとしているのに，javascriptの方で/uploadsとしていた．これにより当然ながらインタフェースがないわけで４０４エラーが返させるということになっていた．分れば馬鹿馬鹿しい勘違いやった😂

あと，デバッグ環境ではルートディレクトリがprojectになるというところもハマった😂

## その４　Speech Recognitionにかける
よし，ここからはpythonの側の処理に集中

In [54]:
%%writefile app04.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)
            
        ai_response = "test"
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app04.py


In [53]:
!python app04.py

5082.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


 * Serving Flask app 'app04'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [27/Feb/2025 06:26:20] "GET / HTTP/1.1" 200 -
マイクテストマイクテストマイクテスト
127.0.0.1 - - [27/Feb/2025 06:26:31] "POST /upload HTTP/1.1" 200 -
マイクテストマイクテスト
127.0.0.1 - - [27/Feb/2025 06:26:43] "POST /upload HTTP/1.1" 200 -
こんにちは初めまして 藤野秀典 と言います
127.0.0.1 - - [27/Feb/2025 06:26:54] "POST /upload HTTP/1.1" 200 -
私の名前は 藤野秀典です あなたの名前は何ですか
127.0.0.1 - - [27/Feb/2025 06:27:06] "POST /upload HTTP/1.1" 200 -
^C


## その５　 openai の　Chat＿Compelationを使う

In [55]:
%%writefile app05.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

        # AIの応答
        client = OpenAI()
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": text},
            ]
        )
        ai_response = completion.choices[0].message.content
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Writing app05.py


In [56]:
!python app05.py

5911.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


 * Serving Flask app 'app05'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 286-038-734
127.0.0.1 - - [27/Feb/2025 06:40:12] "GET / HTTP/1.1" 304 -
こんにちは はじめまして
127.0.0.1 - - [27/Feb/2025 06:40:22] "POST /upload HTTP/1.1" 200 -
^C


ふむ．とりあえずは単発会話はできるようになった．
現時点の違和感・修正したい点は
現状だと，入力テキストとレスポンスが同時に帰ってきてしまう．
どうにか，その部分をいじれないか．レスポンスをまたつに先に入力テキストを返して，画面に表示させておいて，レスポンスが帰ってきたら，改めてそれを返すという感じ．